Importing the libraries

In [1]:
import os
import boto3
import sagemaker

from sagemaker import get_execution_role

region = boto3.Session().region_name

role = get_execution_role()

In [2]:
print (region)

ap-south-1


In [3]:
print (role)

arn:aws:iam::288564358955:role/service-role/AmazonSageMaker-ExecutionRole-20220315T171898


Configuring the bucket for storage

In [4]:
bucket = sagemaker.Session().default_bucket()

In [5]:
bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)

In [6]:
!conda install -y -c conda-forge xgboost==0.90

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/noarch::nbclient==0.5.2=pyhd8ed1ab_0
  - conda-forge/linux-64::matplotlib==3.3.4=py36h5fab9bb_0
  - conda-forge/noarch::qdarkstyle==2.8.1=pyhd8ed1ab_2
  - conda-forge/linux-64::scikit-image==0.16.2=py36hb3f55d8_0
  - conda-forge/noarch::python-language-server==0.36.2=pyhd8ed1ab_0
  - conda-forge/linux-64::widgetsnbextension==3.5.1=py36h5fab9bb_4
  - conda-forge/noarch::flake8==3.8.4=py_0
  - conda-forge/noarch::ipywidgets==7.6.3=pyhd3deb0d_0
  - conda-forge/noarch::typing-extensions==3.7.4.3=0
  - conda-forge/noarch::path.py==12.5.0=0
  - conda-forge/noarch::dask==2021.2.0=pyhd8ed1ab_0
  - conda-forge/noarch::nbformat==5.1.2=pyhd8ed1ab_1
  - conda-forge/linux-64::path==15.1.2=py36h5fab9bb_0
  - conda-forge/linux-64::nbconvert==6.0.7=py36

Name of the model file that we have uploaded on AWS cloud

In [7]:
model_filename = 'DEMO-local-xgboost-model'

In [10]:
import joblib
import xgboost

mymodel = joblib.load(model_filename)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.0.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Upload some test records on AWS cloud and evaluate the model

In [8]:
#import json
import numpy as np

file_name = (
    "test_point.csv"  # customize to your test file, will be 'mnist.single.test' if use data above
)

with open(file_name, "r") as f:
    mypayload = np.loadtxt(f, delimiter=",")
    
print(mypayload)  

[[5.4 3.  4.5 1.5]
 [5.6 3.  4.1 1.3]
 [6.3 2.8 5.1 1.5]
 [6.  3.  4.8 1.8]
 [5.1 3.3 1.7 0.5]]


In [11]:
mymodel.predict(mypayload)

array([1, 1, 1, 2, 0])

In [13]:
mymodel._Booster.save_model(model_filename)

In [14]:
!tar czvf model.tar.gz $model_filename

DEMO-local-xgboost-model


Pushing the model to S3 bucket

In [16]:
#### prefix in S3
prefix = "sagemaker/my-xgboost-byo"

fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, model_filename, "model.tar.gz")
print(key)
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

sagemaker/my-xgboost-byo/DEMO-local-xgboost-model/model.tar.gz


In [17]:
from sagemaker.amazon.amazon_estimator import get_image_uri

#### Get the built-in xgboost container image in Sagemaker to host our model
container = get_image_uri(boto3.Session().region_name, "xgboost", "0.90-1")

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [18]:
%%time
from time import gmtime, strftime

model_name = model_filename + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)

sm_client = boto3.client("sagemaker")

print(model_url)

primary_container = {
    "Image": container,
    "ModelDataUrl": model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response2["ModelArn"])

https://s3-ap-south-1.amazonaws.com/sagemaker-ap-south-1-288564358955/sagemaker/my-xgboost-byo/DEMO-local-xgboost-model/model.tar.gz
arn:aws:sagemaker:ap-south-1:288564358955:model/demo-local-xgboost-model2022-03-15-12-10-00
CPU times: user 76.3 ms, sys: 3.18 ms, total: 79.5 ms
Wall time: 414 ms


Creating the model endpoint on AWS sagemaker

In [19]:
from time import gmtime, strftime

endpoint_config_name = "my-XGBoostEndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

print(endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

my-XGBoostEndpointConfig-2022-03-15-12-10-37
Endpoint Config Arn: arn:aws:sagemaker:ap-south-1:288564358955:endpoint-config/my-xgboostendpointconfig-2022-03-15-12-10-37


In [20]:
%%time
import time

endpoint_name = "my-XGBoostEndpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

my-XGBoostEndpoint-2022-03-15-12-11-49
arn:aws:sagemaker:ap-south-1:288564358955:endpoint/my-xgboostendpoint-2022-03-15-12-11-49
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:ap-south-1:288564358955:endpoint/my-xgboostendpoint-2022-03-15-12-11-49
Status: InService
CPU times: user 61.5 ms, sys: 0 ns, total: 61.5 ms
Wall time: 3min


In [21]:
runtime_client = boto3.client("runtime.sagemaker")

Evaluating the endpoint

In [22]:
%%time
import json


file_name = (
    "test_point.csv"  # customize to your test file, will be 'mnist.single.test' if use data above
)

with open(file_name, "r") as f:
    payload = f.read().strip()
    
    
print("Payload :\n")

print(payload)
print()

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload
)

##print(response)

print("Results :\n")
print()

result = response["Body"].read().decode("ascii")

# Unpack response
print("\nPredicted Class Probabilities: {}.".format(result))

Payload :

5.400000000000000355e+00,3.000000000000000000e+00,4.500000000000000000e+00,1.500000000000000000e+00
5.599999999999999645e+00,3.000000000000000000e+00,4.099999999999999645e+00,1.300000000000000044e+00
6.299999999999999822e+00,2.799999999999999822e+00,5.099999999999999645e+00,1.500000000000000000e+00
6.000000000000000000e+00,3.000000000000000000e+00,4.799999999999999822e+00,1.800000000000000044e+00
5.099999999999999645e+00,3.299999999999999822e+00,1.699999999999999956e+00,5.000000000000000000e-01

Results :



Predicted Class Probabilities: [0.056180261075496674, 0.887361466884613, 0.056458208709955215],[0.056180261075496674, 0.887361466884613, 0.056458208709955215],[0.08053058385848999, 0.8385403752326965, 0.08092901110649109],[0.12150664627552032, 0.27632802724838257, 0.6021653413772583],[0.8827555775642395, 0.0601295568048954, 0.05711490288376808].
CPU times: user 10.6 ms, sys: 3.34 ms, total: 14 ms
Wall time: 144 ms
